In [13]:
pip install google-cloud-videointelligence

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json

from google.cloud import storage

from Annotations_util import get_segments_for_labels

In [2]:
storage_client = storage.Client()

In [43]:
def analyse(event, context):
    '''
    Read annotations
    '''
    #print('event', event)
    
    file_name = event['name']
    bucket_name = event['bucket']
    print(file_name)
    
    # split the fears off the end of the file name name after 🙈 and separate on '&'    
    fears = set(file_name.split('.')[-2].split('🙈')[-1].split('+'))
    print('look for fears', fears)
    fears = set({'bird'})
    
    annotaion_bucket = storage_client.get_bucket(bucket_name)
    
    # downalod annotations from GCS     
    source_blob_name = file_name
    blob = annotaion_bucket.blob(source_blob_name)
    text = blob.download_as_string()
    annotation_results = json.loads(text)
    print(annotation_results)
    
    # print(annotation_results)
    segments = get_segments_for_labels(fears, annotation_results['annotation_results'][0]['shot_label_annotations'])

    
    return segments

    print("\nFinished processing.\n")

In [44]:
event ={'bucket': 'scary-video-annotations', 'name': 'zoo🙈giraffe+rhinoceros.json'}
labels = analyse(event, None)

zoo🙈giraffe+rhinoceros.json
look for fears {'giraffe', 'rhinoceros'}
{'annotation_results': [{'input_uri': '/scary-videos/zoo🙈giraffe+rhinoceros.mp4', 'segment': {'start_time_offset': {}, 'end_time_offset': {'seconds': 140, 'nanos': 920000000}}, 'segment_label_annotations': [{'entity': {'entity_id': '/m/0fbf1m', 'description': 'terrestrial animal', 'language_code': 'en-US'}, 'category_entities': [{'entity_id': '/m/0jbk', 'description': 'animal', 'language_code': 'en-US'}], 'segments': [{'segment': {'start_time_offset': {}, 'end_time_offset': {'seconds': 140, 'nanos': 920000000}}, 'confidence': 0.52990246}]}, {'entity': {'entity_id': '/m/05h0n', 'description': 'nature', 'language_code': 'en-US'}, 'segments': [{'segment': {'start_time_offset': {}, 'end_time_offset': {'seconds': 140, 'nanos': 920000000}}, 'confidence': 0.7760365}]}, {'entity': {'entity_id': '/m/09g5pq', 'description': 'people', 'language_code': 'en-US'}, 'category_entities': [{'entity_id': '/m/01g317', 'description': 'per

In [42]:
annotation_results

NameError: name 'annotation_results' is not defined

In [41]:
%%writefile function_code/Annotations_util.py

def get_time(time_json):
    output = 0
    output += time_json.get('seconds', 0)
    output += time_json.get('nanos', 0) * 1e-9
    return output


def merge_overlaping_segments(sorted_segments):

    merged_segments = []

    for seg in sorted_segments:

        # if its empty add the first one     
        if not merged_segments:
            merged_segments.append(seg)
            continue
        
        # if the start of the next segment is within the previous segment
        # then just extend the previous segment         
        if seg[0] <= merged_segments[-1][1]:
            merged_segments[-1][1] = seg[1]
        else:
            merged_segments.append(seg)

    return merged_segments
    

def get_segments_for_labels(labels, annotations):
    '''
    Find all the segments within the annotations that match any of the labels
    '''
    
    segments = []
    
    for s in annotations:
        
        label_match = False

#         print(s['entity']['description'])
        
        # if the lebel we are looking for matches the entity or the category         
        if s['entity']['description'] in labels:
            label_match = True
        elif 'category_entities' in s:
            if s['category_entities'][0]['description'] in labels:
                label_match = True

        if label_match:
            for seg in s['segments']:
                start_time = get_time( seg['segment']['start_time_offset'])
                end_time = get_time( seg['segment']['end_time_offset'])
                segments.append( [start_time, end_time] )
#                 print('from {} -> {}'.format(start_time, end_time))

    sorted_segments = sorted(segments)
    merged_segments = merge_overlaping_segments(sorted_segments)

    return merged_segments


Writing function_code/Annotations_util.py
